In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import seaborn as sns
#import GMM
from statsmodels.sandbox.regression.gmm import GMM, NonlinearIVGMM, LinearIVGMM
import statsmodels.api as sm
#set seaborn graphics
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [2]:
pathdata = os.getcwd()+'/Data'
os.chdir(pathdata)

In [48]:
unr = pd.read_pickle('un_data.pkl')
gdp = pd.read_pickle('gdp_data.pkl')

In [49]:
i = 0
for df_gdp in [unr, gdp]:
    df_gdp['Current Year'] = df_gdp['Current Year'].astype(float)
    df_gdp['One Year Ahead'] = df_gdp['One Year Ahead'].astype(float)
    df_gdp['Two Years Ahead'] = df_gdp['Two Years Ahead'].astype(float)
    df_gdp['Longer Term'] = df_gdp['Longer Term'].astype(float)
    df_gdp['Rolling1'] = df_gdp['Rolling1'].astype(float)
    df_gdp['Rolling2'] = df_gdp['Rolling2'].astype(float)
    dfgdp1 = df_gdp.sort_values(by=['Source', 'SurveyDate'],ascending = [True, True])
    dfgdp1['CurrentYearDiff'] = dfgdp1['Current Year'].diff()
    dfgdp1['OneYearDiff'] = dfgdp1['One Year Ahead'].diff()
    dfgdp1['LongTermDiff'] = dfgdp1['Longer Term'].diff()
    dfgdp1['TwoYearDiff'] = dfgdp1['Two Years Ahead'].diff()
    dfgdp1['R1'] = dfgdp1['Rolling1'].diff()
    dfgdp1['R2'] = dfgdp1['Rolling2'].diff()
    dfgdp1 = dfgdp1.sort_index()
    df1gdpmeans = dfgdp1.groupby('SurveyDate')[['CurrentYearDiff','OneYearDiff','TwoYearDiff', 'LongTermDiff', 'R1', 'R2']].mean()
    if i == 0:
        unr = df1gdpmeans.copy()
    else:
        gdp = df1gdpmeans.copy()
    i+= 1

In [50]:
unr

,CurrentYearDiff,OneYearDiff,TwoYearDiff,LongTermDiff,R1,R2
SurveyDate,,,,,,
1999-02-12,3.222750,3.022577,NaN,2.228846,3.130591,3.006950
1999-05-03,-0.184464,-0.263571,NaN,NaN,-0.254706,-0.364600
1999-08-06,-0.170217,-0.258913,NaN,NaN,-0.409524,-0.392308
1999-11-03,-0.089302,-0.213810,NaN,NaN,-0.292000,-0.343421
2000-02-09,-0.884681,-0.863191,NaN,NaN,-0.601163,-0.633810
...,...,...,...,...,...,...
2019-01-11,-0.464147,-0.252860,-0.053252,-0.072339,-0.131739,0.003686
2019-03-22,-0.002696,0.023789,0.051803,-0.021695,0.013512,0.042091
2019-07-05,-0.131175,-0.141900,-0.178422,-0.100325,-0.199660,-0.172548


In [51]:
df = unr.merge(gdp, left_index = True, right_index = True, suffixes=('_UNR', '_GDP'))
df

,CurrentYearDiff_UNR,OneYearDiff_UNR,TwoYearDiff_UNR,LongTermDiff_UNR,R1_UNR,R2_UNR,CurrentYearDiff_GDP,OneYearDiff_GDP,TwoYearDiff_GDP,LongTermDiff_GDP,R1_GDP,R2_GDP
SurveyDate,,,,,,,,,,,,
1999-02-12,3.222750,3.022577,NaN,2.228846,3.130591,3.006950,1.043907,1.205866,NaN,1.159783,1.085621,1.270848
1999-05-03,-0.184464,-0.263571,NaN,NaN,-0.254706,-0.364600,-0.092679,0.040714,NaN,NaN,0.176078,0.040588
1999-08-06,-0.170217,-0.258913,NaN,NaN,-0.409524,-0.392308,-0.005000,0.176042,NaN,NaN,0.192619,0.150500
1999-11-03,-0.089302,-0.213810,NaN,NaN,-0.292000,-0.343421,0.052340,0.171064,NaN,NaN,0.405714,0.010000
2000-02-09,-0.884681,-0.863191,NaN,NaN,-0.601163,-0.633810,1.057451,0.155098,NaN,NaN,0.215319,0.096364
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-11,-0.464147,-0.252860,-0.053252,-0.072339,-0.131739,0.003686,-0.507908,-0.303541,-0.165986,-0.035633,-0.263530,-0.184660
2019-03-22,-0.002696,0.023789,0.051803,-0.021695,0.013512,0.042091,-0.335121,-0.103136,-0.000370,-0.035880,-0.181013,-0.105179
2019-07-05,-0.131175,-0.141900,-0.178422,-0.100325,-0.199660,-0.172548,-0.005534,-0.090096,-0.044114,-0.031222,-0.147522,0.023419


In [60]:
df.index = pd.to_datetime(df.index)
oisfactors = pd.read_csv('dataset_con.csv')
oisfactors.index = pd.to_datetime(oisfactors['Date'])


#End of period DEFINITION - always take the last shock
mpdf = pd.read_excel('Dataset_EA-MPD.xlsx', sheet_name='Monetary Event Window')
mpdf.index = mpdf['date']
prdf = pd.read_excel('Dataset_EA-MPD.xlsx', sheet_name='Press Release Window')
prdf.index = prdf['date']
pcdf = pd.read_excel('Dataset_EA-MPD.xlsx', sheet_name='Press Conference Window')
pcdf.index = pcdf['date']
pcdf = pd.concat([pcdf, oisfactors], axis = 1)
##get OLS estimate of the OIS 3Y
pcdf1 = pcdf.loc[(pcdf.index>'2001-12-01')&(pcdf.index<'2015-01-01')]
pcdf1 = pcdf1.dropna(subset=['OIS_1M', 'OIS_3M', 'OIS_6M', 'OIS_1Y', 'OIS_2Y', 'OIS_3Y'])
X = sm.add_constant(pcdf1[['OIS_1M', 'OIS_3M', 'OIS_6M', 'OIS_1Y']])
model = sm.OLS(pcdf1['OIS_3Y'], X)
results = model.fit()
#get the residuals and the fitted values
oisnew = pd.DataFrame(results.resid)
oisnew.columns = ['oisnew']
oisnew.index = pcdf1.index
oisnewfitted = pd.DataFrame(results.fittedvalues)
oisnewfitted.columns = ['oisnew_fitted']
oisnewfitted.index = pcdf1.index
pcdf = pd.concat([pcdf, oisnew], axis = 1)
pcdf = pd.concat([pcdf, oisnewfitted], axis = 1)

dates_list = df.index.unique()
date1=dates_list[0]
date2=dates_list[1]
newdf = mpdf.loc[(mpdf.index<date1)].tail(1)
#tale always the last announcement
for i in range(0, len(dates_list)-1):
    newdfwork = mpdf.loc[(mpdf.index>dates_list[i])&(mpdf.index<dates_list[i+1])].tail(1)
    #print(newdfwork)
    newdf = newdf.append(newdfwork)
newdf.index = dates_list
#repeat same for press conference
dates_list = df.index.unique()
date1=dates_list[0]
date2=dates_list[1]
newdfpr = prdf.loc[(prdf.index<date1)].tail(1)
for i in range(0, len(dates_list)-1):
    newdfpr = newdfpr.append(prdf.loc[(prdf.index>dates_list[i])&(prdf.index<dates_list[i+1])].tail(1))
newdfpr.index = dates_list

#repeat same for press release
dates_list = df.index.unique()
date1=dates_list[0]
date2=dates_list[1]
newdfpc = pcdf.loc[(pcdf.index<date1)].tail(1)
for i in range(0, len(dates_list)-1):
    newdfpc = newdfpc.append(pcdf.loc[(pcdf.index>dates_list[i])&(pcdf.index<dates_list[i+1])].tail(1))
newdfpc.index = dates_list

#SUM DEFINITION
#mpdf = pd.read_excel('Dataset_EA-MPD.xlsx', sheet_name='Monetary Event Window')
#mpdf.index = mpdf['date']
#prdf = pd.read_excel('Dataset_EA-MPD.xlsx', sheet_name='Press Release Window')
#prdf.index = prdf['date']
#pcdf = pd.read_excel('Dataset_EA-MPD.xlsx', sheet_name='Press Conference Window')
#pcdf.index = pcdf['date']
dates_list = df.index.unique()
date1=dates_list[0]
date2=dates_list[1]
newdfsum = mpdf.loc[(mpdf.index<date1)].tail(1)
for i in range(0, len(dates_list)-1):
    newdfworksum = mpdf.loc[(mpdf.index>dates_list[i])&(mpdf.index<dates_list[i+1])].sum()
    newdfsum = newdfsum.append(newdfwork, ignore_index = True)
newdfsum.index = dates_list
#repeat same for press conference
dates_list = df.index.unique()
date1=dates_list[0]
date2=dates_list[1]
newdfprsum = prdf.loc[(prdf.index<date1)].tail(1)
for i in range(0, len(dates_list)-1):
    newdfprsum = newdfprsum.append(prdf.loc[(prdf.index>dates_list[i])&(prdf.index<dates_list[i+1])].sum(), ignore_index = True)
newdfprsum.index = dates_list

#repeat same for press release
dates_list = df.index.unique()
date1=dates_list[0]
date2=dates_list[1]
newdfpcsum = pcdf.loc[(pcdf.index<date1)].tail(1)
for i in range(0, len(dates_list)-1):
    newdfpcsum = newdfpcsum.append(pcdf.loc[(pcdf.index>dates_list[i])&(pcdf.index<dates_list[i+1])].sum(),ignore_index = True)
newdfpcsum.index = dates_list

In [75]:
'''
'''
class mpGMM(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        # TODO: clean up signature
        kwds.setdefault('k_moms', 20)
        kwds.setdefault('k_params', 16)
        super(mpGMM, self).__init__(*args, **kwds)


    def momcond(self, params):
        c0, c1, c2, c3, c4, c5, c6, c7, c0_unr, c1_unr, c2_unr, c3_unr, c4_unr, c5_unr, c6_unr, c7_unr = params
        endog = self.endog
        exog = self.exog
        #defining variables
        #interaction dummies
        dQ1 = exog[:,0]
        dQ2 = exog[:,1]
        dQ3 = exog[:,2]
        dQ4 = exog[:,3]
        dTY = exog[:,4]
        #endogenous variables
        currentY_gdp = endog[:,0]
        nextY_gdp = endog[:,1]
        twoY_gdp = endog[:,2]
        rolling1_gdp = endog[:,3]
        rolling2_gdp = endog[:,4]
        currentY_unr = endog[:,5]
        nextY_unr = endog[:,6]
        rolling1_unr = endog[:,7]
        rolling2_unr = endog[:,8]        
        #monetary policy shock
        mp = exog[:,5]
        #next year forecasts GDP
        error1 = dQ1*mp*(nextY_gdp-mp*(1/4*c1+1/2*c2+3/4*c3+1*c4+3/4*c5+1/2*c6+1/4*c7)*dQ1)
        error2 = dQ2*mp*(nextY_gdp-mp*(1/4*c0+1/2*c1+3/4*c2+1*c3+3/4*c4+1/2*c5+1/4*c6)*dQ2)
        error3 = dQ3*mp*(nextY_gdp-mp*(1/2*c0+3/4*c1+1*c2+3/4*c3+1/2*c4+1/4*c5)*dQ3)
        error4 = dQ4*mp*(nextY_gdp-mp*(3/4*c0+1*c1+3/4*c2+1/2*c3+1/4*c4)*dQ4)
        #next year forecasts UNR
        error5 = dQ1*mp*(nextY_unr-mp*(1/4*c1_unr+1/2*c2_unr+3/4*c3_unr+1*c4_unr+3/4*c5_unr+1/2*c6_unr+1/4*c7_unr)*dQ1)
        error6 = dQ2*mp*(nextY_unr-mp*(1/4*c0_unr+1/2*c1_unr+3/4*c2_unr+1*c3_unr+3/4*c4_unr+1/2*c5_unr+1/4*c6_unr)*dQ2)
        error7 = dQ3*mp*(nextY_unr-mp*(1/2*c0_unr+3/4*c1_unr+1*c2_unr+3/4*c3_unr+1/2*c4_unr+1/4*c5_unr)*dQ3)
        error8 = dQ4*mp*(nextY_unr-mp*(3/4*c0_unr+1*c1_unr+3/4*c2_unr+1/2*c3_unr+1/4*c4_unr)*dQ4)
        #current year forecasts
        error9 = dQ1*mp*(currentY_gdp-mp*(c0+3/4*c1+1/2*c2+1/4*c3)*dQ1)
        error10 = dQ2*mp*(currentY_gdp-mp*(3/4*c0+1/2*c1+1/4*c2)*dQ2)
        error11 = dQ3*mp*(currentY_gdp-mp*(1/2*c0+1/4*c1)*dQ3)
        error12 = dQ4*mp*(currentY_gdp-mp*(1/4*c0)*dQ4)
        #current year forecasts
        error13 = dQ1*mp*(currentY_unr-mp*(c0_unr+3/4*c1_unr+1/2*c2_unr+1/4*c3_unr)*dQ1)
        error14 = dQ2*mp*(currentY_unr-mp*(3/4*c0_unr+1/2*c1_unr+1/4*c2_unr)*dQ2)
        error15 = dQ3*mp*(currentY_unr-mp*(1/2*c0_unr+1/4*c1_unr)*dQ3)
        error16 = dQ4*mp*(currentY_unr-mp*(1/4*c0_unr)*dQ4)        
        #rolling forecasts
        #r
        error17 = mp*(rolling1_gdp-mp*(c2))
        error18 = mp*(rolling2_gdp-mp*(c6))
        #rolling forecasts
        #r
        error19 = mp*(rolling1_unr-mp*(c2_unr))
        error20 = mp*(rolling2_unr-mp*(c6_unr))
        g = np.column_stack((error1, error2, error3, error4, error5, error6, error7, error8, error9, error10, 
                            error11, error12, error13, error14, error15, error16, error17, error18, error19, error20))
        return g

#order data
dataols = pd.concat([df, newdfpc], axis=1)
var = 'OIS_2Y'
dataols = dataols.dropna(subset = [var]).iloc[:-1,:]
#time periods
dataols = dataols.loc[(dataols.index>'2001-12-01')&(dataols.index<'2015-01-01')]
#dataols = dataols.loc[dataols.index>'2002-01-01']
#create dummies
dataols['D1'] = (dataols.index.month.isin([1,2])==True)*1
dataols['D2'] = (dataols.index.month.isin([3,4])==True)*1
dataols['D3'] = (dataols.index.month.isin([7,8])==True)*1
dataols['D4'] = (dataols.index.month.isin([10,11])==True)*1
dataols['DTY'] = (dataols['TwoYearDiff_GDP'].isnull()==False)*1
dataols['TwoYearDiff_GDP'] = dataols['TwoYearDiff_GDP'].fillna(value=0)
exog = dataols[['D1', 'D2', 'D3', 'D4', 'DTY', var]]
#exog = dataols[[var]]
endog = dataols[['CurrentYearDiff_GDP', 'OneYearDiff_GDP', 'TwoYearDiff_GDP', 'R1_GDP', 'R2_GDP',
                'CurrentYearDiff_UNR', 'OneYearDiff_UNR', 'R1_UNR', 'R2_UNR']].copy()
#instr = np.asarray(exogt.iloc[:, 4:])
model = mpGMM(endog, exog,  None)
beta0 = np.zeros((1, 16))
res = model.fititer(beta0, maxiter=100, optim_method='nm',   weights_method='cov',
                  wargs={'centered':False})
#,  weights_method='cov',
#                    wargs={'centered':False}
w0 = res[1]
inv_weights = np.linalg.inv(w0)
#inv_weights =inv_weights,
res = model.fit(beta0, maxiter=10,   inv_weights =inv_weights, optim_method='nm',   weights_method='cov',
                   wargs={'centered':False})
print(res.summary(yname='GDP forecast diff', xname=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7',
                                                   'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15']))

Optimization terminated successfully.
         Current function value: 0.145610
         Iterations: 2233
         Function evaluations: 3002
Optimization terminated successfully.
         Current function value: 0.140768
         Iterations: 1437
         Function evaluations: 1926
Optimization terminated successfully.
         Current function value: 0.134555
         Iterations: 1459
         Function evaluations: 1974
Optimization terminated successfully.
         Current function value: 0.125337
         Iterations: 1686
         Function evaluations: 2274
Optimization terminated successfully.
         Current function value: 0.118577
         Iterations: 1968
         Function evaluations: 2647
Optimization terminated successfully.
         Current function value: 0.115844
         Iterations: 1184
         Function evaluations: 1618
Optimization terminated successfully.
         Current function value: 0.112935
         Iterations: 2316
         Function evaluations: 3128
Optimi

Optimization terminated successfully.
         Current function value: 0.114104
         Iterations: 210
         Function evaluations: 375
Optimization terminated successfully.
         Current function value: 0.114105
         Iterations: 210
         Function evaluations: 375
Optimization terminated successfully.
         Current function value: 0.114100
         Iterations: 222
         Function evaluations: 389
Optimization terminated successfully.
         Current function value: 0.114111
         Iterations: 232
         Function evaluations: 407
Optimization terminated successfully.
         Current function value: 0.114108
         Iterations: 209
         Function evaluations: 370
Optimization terminated successfully.
         Current function value: 0.114112
         Iterations: 183
         Function evaluations: 338
Optimization terminated successfully.
         Current function value: 0.114102
         Iterations: 205
         Function evaluations: 369
Optimization termina

In [84]:
rmatrix = np.array([[1, 0, 0, 0, 0, 0, 0, 0, -1.8, 0, 0, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 0, 0, 0, 0, 0, -1.8, 0, 0, 0, 0, 0, 0],
                    [0, 0, 1, 0, 0, 0, 0, 0, 0,  0,-1.8, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0,  0,-1.8, 0, 0, 0, 0],
                    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,  0,-1.8, 0, 0, 0],
                    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1.8, 0, 0],
                    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1.8, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1.8]])

In [85]:
res.t_test(rmatrix)

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.2253      0.038     -5.999      0.000      -0.299      -0.152
c1             0.3126      0.064      4.857      0.000       0.186       0.439
c2            -0.0306      0.017     -1.778      0.075      -0.064       0.003
c3            -0.4974      0.364     -1.365      0.172      -1.212       0.217
c4             0.4941      0.721      0.685      0.493      -0.919       1.907
c5             0.0822      0.427      0.193      0.847      -0.754       0.918
c6            -0.0169      0.010     -1.648      0.099      -0.037       0.003
c7            -1.1408      0.970     -1.176      0.240      -3.043       0.761

In [86]:
rmatrix.shape

(8, 16)

In [87]:
'''
'''
class mpGMM(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        # TODO: clean up signature
        kwds.setdefault('k_moms', 20)
        kwds.setdefault('k_params', 8)
        super(mpGMM, self).__init__(*args, **kwds)


    def momcond(self, params):
        c0, c1, c2, c3, c4, c5, c6, c7  = params
        endog = self.endog
        exog = self.exog
        #defining variables
        #interaction dummies
        dQ1 = exog[:,0]
        dQ2 = exog[:,1]
        dQ3 = exog[:,2]
        dQ4 = exog[:,3]
        dTY = exog[:,4]
        #endogenous variables
        currentY_gdp = endog[:,0]
        nextY_gdp = endog[:,1]
        twoY_gdp = endog[:,2]
        rolling1_gdp = endog[:,3]
        rolling2_gdp = endog[:,4]
        currentY_unr = endog[:,5]
        nextY_unr = endog[:,6]
        rolling1_unr = endog[:,7]
        rolling2_unr = endog[:,8]        
        #monetary policy shock
        mp = exog[:,5]
        #next year forecasts GDP
        error1 = dQ1*mp*(nextY_gdp-mp*(1/4*c1+1/2*c2+3/4*c3+1*c4+3/4*c5+1/2*c6+1/4*c7)*dQ1)
        error2 = dQ2*mp*(nextY_gdp-mp*(1/4*c0+1/2*c1+3/4*c2+1*c3+3/4*c4+1/2*c5+1/4*c6)*dQ2)
        error3 = dQ3*mp*(nextY_gdp-mp*(1/2*c0+3/4*c1+1*c2+3/4*c3+1/2*c4+1/4*c5)*dQ3)
        error4 = dQ4*mp*(nextY_gdp-mp*(3/4*c0+1*c1+3/4*c2+1/2*c3+1/4*c4)*dQ4)
        #next year forecasts UNR
        error5 = dQ1*mp*(nextY_unr-mp*(1/4*c1*-1.8+1/2*c2*-1.8+3/4*c3*-1.8+1*c4*-1.8+3/4*c5*-1.8+1/2*c6*-1.8+1/4*c7*-1.8)*dQ1)
        error6 = dQ2*mp*(nextY_unr-mp*(1/4*c0*-1.8+1/2*c1*-1.8+3/4*c2*-1.8+1*c3*-1.8+3/4*c4*-1.8+1/2*c5*-1.8+1/4*c6*-1.8)*dQ2)
        error7 = dQ3*mp*(nextY_unr-mp*(1/2*c0*-1.8+3/4*c1*-1.8+1*c2*-1.8+3/4*c3*-1.8+1/2*c4*-1.8+1/4*c5*-1.8)*dQ3)
        error8 = dQ4*mp*(nextY_unr-mp*(3/4*c0*-1.8+1*c1*-1.8+3/4*c2*-1.8+1/2*c3*-1.8+1/4*c4*-1.8)*dQ4)
        #current year forecasts
        error9 = dQ1*mp*(currentY_gdp-mp*(c0+3/4*c1+1/2*c2+1/4*c3)*dQ1)
        error10 = dQ2*mp*(currentY_gdp-mp*(3/4*c0+1/2*c1+1/4*c2)*dQ2)
        error11 = dQ3*mp*(currentY_gdp-mp*(1/2*c0+1/4*c1)*dQ3)
        error12 = dQ4*mp*(currentY_gdp-mp*(1/4*c0)*dQ4)
        #current year forecasts
        error13 = dQ1*mp*(currentY_unr-mp*(c0*-1.8+3/4*c1*-1.8+1/2*c2*-1.8+1/4*c3*-1.8)*dQ1)
        error14 = dQ2*mp*(currentY_unr-mp*(3/4*c0*-1.8+1/2*c1*-1.8+1/4*c2*-1.8)*dQ2)
        error15 = dQ3*mp*(currentY_unr-mp*(1/2*c0*-1.8+1/4*c1*-1.8)*dQ3)
        error16 = dQ4*mp*(currentY_unr-mp*(1/4*c0*-1.8)*dQ4)        
        #rolling forecasts
        #r
        error17 = mp*(rolling1_gdp-mp*(c2))
        error18 = mp*(rolling2_gdp-mp*(c6))
        #rolling forecasts
        #r
        error19 = mp*(rolling1_unr-mp*(c2*-1.8))
        error20 = mp*(rolling2_unr-mp*(c6*-1.8))
        g = np.column_stack((error1, error2, error3, error4, error5, error6, error7, error8, error9, error10, 
                            error11, error12, error13, error14, error15, error16, error17, error18, error19, error20))
        return g

#order data
dataols = pd.concat([df, newdfpc], axis=1)
var = 'OIS_2Y'
dataols = dataols.dropna(subset = [var]).iloc[:-1,:]
#time periods
dataols = dataols.loc[(dataols.index>'2001-12-01')&(dataols.index<'2015-01-01')]
#dataols = dataols.loc[dataols.index>'2002-01-01']
#create dummies
dataols['D1'] = (dataols.index.month.isin([1,2])==True)*1
dataols['D2'] = (dataols.index.month.isin([3,4])==True)*1
dataols['D3'] = (dataols.index.month.isin([7,8])==True)*1
dataols['D4'] = (dataols.index.month.isin([10,11])==True)*1
dataols['DTY'] = (dataols['TwoYearDiff_GDP'].isnull()==False)*1
dataols['TwoYearDiff_GDP'] = dataols['TwoYearDiff_GDP'].fillna(value=0)
exog = dataols[['D1', 'D2', 'D3', 'D4', 'DTY', var]]
#exog = dataols[[var]]
endog = dataols[['CurrentYearDiff_GDP', 'OneYearDiff_GDP', 'TwoYearDiff_GDP', 'R1_GDP', 'R2_GDP',
                'CurrentYearDiff_UNR', 'OneYearDiff_UNR', 'R1_UNR', 'R2_UNR']].copy()
#instr = np.asarray(exogt.iloc[:, 4:])
model = mpGMM(endog, exog,  None)
beta0 = np.zeros((1, 8))
res = model.fititer(beta0, maxiter=100, optim_method='nm',   weights_method='cov',
                  wargs={'centered':False})
#,  weights_method='cov',
#                    wargs={'centered':False}
w0 = res[1]
inv_weights = np.linalg.inv(w0)
#inv_weights =inv_weights,
res = model.fit(beta0, maxiter=10,   inv_weights =inv_weights, optim_method='nm',   weights_method='cov',
                   wargs={'centered':False})
print(res.summary(yname='GDP forecast diff', xname=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']))

Optimization terminated successfully.
         Current function value: 0.258888
         Iterations: 460
         Function evaluations: 698
Optimization terminated successfully.
         Current function value: 0.262367
         Iterations: 420
         Function evaluations: 638
Optimization terminated successfully.
         Current function value: 0.264300
         Iterations: 366
         Function evaluations: 560
Optimization terminated successfully.
         Current function value: 0.265443
         Iterations: 178
         Function evaluations: 288
Optimization terminated successfully.
         Current function value: 0.266012
         Iterations: 203
         Function evaluations: 323
Optimization terminated successfully.
         Current function value: 0.266351
         Iterations: 183
         Function evaluations: 294
Optimization terminated successfully.
         Current function value: 0.266574
         Iterations: 155
         Function evaluations: 242
Optimization termina

Optimization terminated successfully.
         Current function value: 0.255944
         Iterations: 207
         Function evaluations: 327
Optimization terminated successfully.
         Current function value: 0.251062
         Iterations: 492
         Function evaluations: 743
Optimization terminated successfully.
         Current function value: 0.253368
         Iterations: 86
         Function evaluations: 158
Optimization terminated successfully.
         Current function value: 0.253612
         Iterations: 188
         Function evaluations: 298
Optimization terminated successfully.
         Current function value: 0.253728
         Iterations: 88
         Function evaluations: 165
Optimization terminated successfully.
         Current function value: 0.253837
         Iterations: 82
         Function evaluations: 161
Optimization terminated successfully.
         Current function value: 0.253908
         Iterations: 101
         Function evaluations: 184
Optimization terminated

In [107]:
datesecb = pd.read_excel('ecb_dates.xlsx')

In [108]:
datesecb.index = datesecb['ECB forecast']

In [109]:
ecbgdp = pd.read_pickle('ecbdiffgdp.pkl')

In [110]:
ecbgdp = pd.concat([ecbgdp, datesecb], axis = 1)

In [111]:
ecbgdp = ecbgdp.dropna(subset=['Cut-off'])

In [112]:
ecbgdp.index = ecbgdp['Cut-off']

In [113]:
ecbgdp = ecbgdp.drop(['2Y','ECB forecast', 'Cut-off'], axis = 1)

In [114]:
ecbgdp.to_pickle('ecbnew.pkl')